In [1]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math

# Load Data

In [2]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #Is transformed to be better with VGG16 model
    ])

In [3]:
def getDataset(train=True, sample_size=100):
    dataset = CIFAR10(root='./data',
                  train=train, 
                  download=True,
                  transform=transform)
    
    X = []
    y = []

    for i in range(sample_size):
        X.append( dataset[i][0] )
        y.append( dataset[i][1] )

    X = torch.stack( X )

    return X, y

X_train, y_train = getDataset() #Training data
X_test, y_test = getDataset(train=False) #Test data

Files already downloaded and verified
Files already downloaded and verified


# Load Model

In [4]:
from torchvision import models
pretrained_model = models.vgg16(pretrained=True)
pretrained_model.classifier = torch.nn.Identity() # 25088 features output

c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Test\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(25088, 10000),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(10000, 4000)    # Second fully connected layer to reduce to 4000
        )

    def forward(self, x):
        return self.fc_layers(x)

In [6]:
class CombinedModel(nn.Module):
    def __init__(self, cnn_extractor, custom_nn):
        super(CombinedModel, self).__init__()
        self.cnn_extractor = cnn_extractor  # Pretrained CNN (frozen)
        self.custom_nn = custom_nn  # Custom NN (trainable)

    def forward(self, x):
        features = self.cnn_extractor(x)  # Extract features using the CNN
        output = self.custom_nn(features)  # Pass the features through the custom NN
        return output


In [7]:
custom_nn = CustomNN()
model = CombinedModel(pretrained_model, custom_nn)

## Custom Loss Function

In [15]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self, mean, std_dev, eta = 0.01, hash_length = 32):
        super(CustomLoss, self).__init__()
        # Initialize W and V as learnable parameters
        self.W = nn.Parameter(torch.normal(mean, std_dev, (4000, hash_length)))  # W will be updated during training
        self.V = nn.Parameter(torch.normal(mean, std_dev, (hash_length, 1)))     # V will be updated during training
        self.eta = eta  # Regularization parameter

    def forward(self, outputs, targets, hash_length = 32):
        targets = torch.tensor(targets)
        S = (targets[:, None] == targets).float() # S calculation

        print(S.shape)

        #U calculation
        U = []
        for i in range(len(outputs)):
            dot = torch.matmul(self.W.T, outputs[i])
            dot = dot.reshape(hash_length, 1)
            u = (dot + self.V)
            U.append(u)
        U = torch.stack(U) # torch.Size([100, 32, 1])
        U = U.reshape(100, hash_length)
        print(U.shape)
        #Calculate Theta
        dot_product_matrix = torch.matmul(U, U.T)
        dot_product_matrix # (sample_size, sample_size) Shape
        Theta = 1/2 * dot_product_matrix

        #Calculate hash codes
        B = torch.sign(U)
        print(B.shape)
        loss = - torch.sum(S * Theta - torch.log(1 + torch.exp(Theta))) + self.eta * torch.sum(torch.norm(B - U, dim = 1).pow(2))   
        print(loss)
        return loss.mean()


# Using Custom Loss Function

In [9]:
num_epochs = 5

In [16]:
loss_fn = CustomLoss(mean=0, std_dev=0.01, eta=0.25)


for param in pretrained_model.parameters(): #Lås parameters i VGG16
    param.requires_grad = False

optimizer = optim.Adam(model.custom_nn.parameters(), lr=0.001)


for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients
    inputs = X_train  # Your input data
    targets = y_train  # Your target labels
    outputs = model(inputs)  # Forward pass through the combined model
    loss = loss_fn(outputs, targets)  # Compute the custom loss
    #print(len(outputs))
    #print(len(targets))
    loss.backward()  # Backpropagate
    optimizer.step()  # Update only the custom NN's parameters

torch.Size([100, 100])
torch.Size([100, 32])
torch.Size([100, 32])
tensor(nan, grad_fn=<AddBackward0>)
torch.Size([100, 100])
torch.Size([100, 32])
torch.Size([100, 32])
tensor(nan, grad_fn=<AddBackward0>)
torch.Size([100, 100])
torch.Size([100, 32])
torch.Size([100, 32])
tensor(nan, grad_fn=<AddBackward0>)
torch.Size([100, 100])
torch.Size([100, 32])
torch.Size([100, 32])
tensor(nan, grad_fn=<AddBackward0>)


KeyboardInterrupt: 